In [ ]:
# Downloads required packages and files
required_files = "https://github.com/jhu-intro-hlt/jhu-intro-hlt.github.io/raw/master/assignments/hw4-files/student/required_files.zip"
! wget $required_files && unzip -o required_files.zip


--2023-12-19 02:26:24--  https://github.com/jhu-intro-hlt/jhu-intro-hlt.github.io/raw/master/assignments/hw4-files/student/required_files.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/jhu-intro-hlt/jhu-intro-hlt.github.io/master/assignments/hw4-files/student/required_files.zip [following]
--2023-12-19 02:26:25--  https://raw.githubusercontent.com/jhu-intro-hlt/jhu-intro-hlt.github.io/master/assignments/hw4-files/student/required_files.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2395 (2.3K) [application/zip]
Saving to: ‘required_files.zip’

required_files.zip  100%[=================

The required.txt file needs to be modified with 2 things:
1) sklearn needs to be replaced to be scikit-learn
2) torchmetrics should be replaced by torchmetrics == 0.11.4

In [ ]:
! pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.9/166.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 708.1/708.1 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.1/160.1 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 73.9 MB/s eta 0:00:00
  Created wheel for wkhtmltopdf: filename=wkhtmltopdf-0.2-py3-none-any.whl size=11134 sha256=bb4c70a017038167d6484ac46633bd6c94171bc5f7566cf323

In [ ]:
# Initialize Otter
import otter

grader = otter.Notebook(colab=True)



```
# Paste your Colab notebook link here
```



First let's set up some helper code and import the libraries which we will use in this experiment.

In [ ]:
import os
from typing import Dict, List, Tuple, Any, Optional, Union
from dataclasses import dataclass

import torch
import torchaudio
import torch.nn as nn
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Dataset
from transformers.modeling_outputs import CausalLMOutput

try:
    from vocabulary import Vocabulary, encode_as_tensor, decode_as_str
except:
    from files.vocabulary import Vocabulary, encode_as_tensor, decode_as_str

In [ ]:
# Checks whether it is in the autograder grading mode
# Checks whether GPU accelerators are available
is_autograder = os.path.exists('is_autograder.py')
if torch.cuda.is_available() and not is_autograder:
    accelerator = 'gpu'
elif os.environ.get('COLAB_TPU_ADDR') is not None and not is_autograder:
    accelerator = 'tpu'
else:
    accelerator = 'cpu'
print(f'The notebook is running for "{"autograder" if is_autograder else "student"}".')
print('Students should make sure you are running under the "student" mode.')
print(f'You are using "{accelerator}".')

The notebook is running for "student".
Students should make sure you are running under the "student" mode.
You are using "gpu".


In [ ]:
# Seed everything to make sure all experiments are reproducible
pl.seed_everything(seed=777)

INFO:pytorch_lightning.utilities.seed:Global seed set to 777


777

In [ ]:
# Defines constants
DATA_URL = "https://github.com/jhu-intro-hlt/jhu-intro-hlt.github.io/raw/master/assignments/hw4-files/student/"

In [ ]:
# Attention!!!
# Set this to True, if you are using local machine instead of Colab
RUN_LOCALLY = False
# But you have to make sure that your machine has CUDA support
# Otherwise the training would be super slow.
LOCAL_DATA_STORE_PATH = 'librispeech_data'  # If run locally, please configure a path to store the dataset.
# Only try to use the Colab and Google Drive when not under the autograder environment
if not is_autograder:
    if not RUN_LOCALLY:
        try:
            LIBRISPEECH_DATA_PATH = './gdrive/My Drive/librispeech_data'
            from google.colab import drive

            drive.mount('/content/gdrive')
        except:  # Fall back to local storage
            LIBRISPEECH_DATA_PATH = LOCAL_DATA_STORE_PATH
    else:
        LIBRISPEECH_DATA_PATH = LOCAL_DATA_STORE_PATH
else:
    LIBRISPEECH_DATA_PATH = LOCAL_DATA_STORE_PATH
os.makedirs(LIBRISPEECH_DATA_PATH, exist_ok=True)

Mounted at /content/gdrive


In [ ]:
if not is_autograder:
    train_dataset = torchaudio.datasets.LIBRISPEECH(LIBRISPEECH_DATA_PATH, url='train-clean-100', download=True)
    dev_dataset = torchaudio.datasets.LIBRISPEECH(LIBRISPEECH_DATA_PATH, url='dev-clean', download=True)
else:
    train_dataset = None
    dev_dataset = None

# Vocabulary



In [ ]:
char_to_map = char_map_str = """'
a
b
c
d
e
f
g
h
i
j
k
l
m
n
o
p
q
r
s
t
u
v
w
x
y
z"""

vocab = Vocabulary()
for c in char_to_map.split('\n'):
    vocab.add_token(token=c.strip())

## Downloading the data
We will use a 100h subset of LibrSpeech to train our models. Run the code cell below to download the training and testing data.


## Feature extraction



_Type your answer here, replacing this text._

In [ ]:


train_audio_transforms = nn.Sequential(
     torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=64),
     torchaudio.transforms.FrequencyMasking(freq_mask_param=30),
     torchaudio.transforms.TimeMasking(time_mask_param=100)
)

# Feature extraction for test data. Use the same sample rate and number of mels
# as the training data. No SpecAugment is needed here.

valid_audio_transforms =  torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=64)

In [ ]:
@dataclass
class ASRBatch:
    spectrograms: torch.Tensor  # (batch_size, channel, feature, time)
    labels: torch.Tensor  # (batch_size, label_len, num_labels)
    attention_mask: torch.Tensor  # (batch_size, time)
    label_mask: torch.Tensor  # (batch_size, label_len)
    label_strs: List[str]


class SpeechRecognitionDataModule(pl.LightningDataModule):
    """Wraps PyTorch dataset as a lightning data module."""

    def __init__(
            self,
            datasets: Dict[str, Dataset],
            vocab: Vocabulary,
            batch_size: int = 32,
            shuffle: bool = True
    ):
        super(SpeechRecognitionDataModule, self).__init__()

        self.datasets: Dict[str, Dataset] = {k: v for k, v in datasets.items() if v is not None}
        self.vocab = vocab
        self.batch_size = batch_size
        self.shuffle = shuffle

    @staticmethod
    def _pad_sequence(seq: torch.Tensor, max_length: int, padding_value: Union[int, float, bool]) -> torch.Tensor:
        seq_len = seq.shape[-1]
        if seq_len < max_length:
            return torch.cat(
                [seq, torch.tensor([padding_value] * (max_length - seq_len), dtype=seq.dtype, device=seq.device)],
                dim=-1
            )
        else:
            return seq

    def collate_fn(self, data, phase='train'):
        spectrograms = []
        labels = []
        label_strs = []
        input_lengths = []
        label_lengths = []
        for (waveform, _, utterance, _, _, _) in data:
            if phase == 'train':
                spec = train_audio_transforms(waveform).squeeze(0).transpose(0, 1)
            elif phase == 'val' or phase == 'test':
                spec = valid_audio_transforms(waveform).squeeze(0).transpose(0, 1)
            else:
                raise ValueError
            spectrograms.append(spec)
            label_str = utterance.lower()
            label = encode_as_tensor(self.vocab, label_str).squeeze(0)
            labels.append(label)
            label_strs.append(label_str)
            input_lengths.append(spec.shape[0] // 2)
            label_lengths.append(label.shape[-1])

        spectrograms = nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).unsqueeze(1).transpose(2, 3)
        labels = nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=self.vocab.pad_id())

        max_input_len = max(input_lengths)
        max_label_len = max(label_lengths)
        attention_mask = torch.stack(
            [
                self._pad_sequence(torch.ones(l, dtype=torch.bool), max_length=max_input_len, padding_value=False)
                for l in input_lengths
            ],
            dim=0
        )
        label_mask = torch.stack(
            [
                self._pad_sequence(torch.ones(l, dtype=torch.bool), max_length=max_label_len, padding_value=False)
                for l in label_lengths
            ],
            dim=0
        )

        return ASRBatch(
            spectrograms=spectrograms,
            labels=labels,
            attention_mask=attention_mask,
            label_mask=label_mask,
            label_strs=label_strs
        )

    def train_dataloader(self):
        return DataLoader(self.datasets['train'],
                          batch_size=self.batch_size,
                          shuffle=self.shuffle,
                          collate_fn=lambda x: self.collate_fn(x, phase='train'))

    def val_dataloader(self):
        return DataLoader(self.datasets['val'],
                          batch_size=self.batch_size,
                          shuffle=False,
                          collate_fn=lambda x: self.collate_fn(x, phase='val'))

    def test_dataloader(self):
        return DataLoader(self.datasets['test'],
                          batch_size=self.batch_size,
                          shuffle=False,
                          collate_fn=lambda x: self.collate_fn(x, phase='test'))

In [ ]:
test_datamodule = SpeechRecognitionDataModule(
    datasets={
        'train': train_dataset,
        'val': dev_dataset
    },
    vocab=vocab,
    batch_size=4
)

In [ ]:
train_dataloader = test_datamodule.train_dataloader()
for i, bc in enumerate(train_dataloader):
    if i > 0:
        break
    print(bc)

ASRBatch(spectrograms=tensor([[[[2.8718e-03, 5.1259e-03, 5.1706e-03,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [3.1303e-03, 6.7686e-03, 5.8033e-03,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [3.6288e-03, 1.0869e-02, 7.1558e-03,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          ...,
          [9.9442e-04, 1.0929e-03, 2.1029e-03,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [6.6392e-04, 9.4669e-04, 1.6397e-03,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [5.3676e-04, 4.2598e-04, 5.0813e-04,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00]]],


        [[[1.2488e-01, 1.6741e-02, 4.3402e-01,  ..., 1.3716e+00,
           1.3200e+00, 5.5934e-01],
          [1.0877e-01, 1.3796e-02, 3.4552e-01,  ..., 1.0400e+00,
           9.9259e-01, 4.2907e-01],
          [5.6131e-02, 4.6071e-03, 7.4245e-02,  ..., 4.1466e-02,
           9.1925e-03, 3.5386e-02],
          ...,
          [5.5554e-04, 5.0497e-04, 

In [ ]:
import torch
import torch.nn as nn
from typing import Optional
from transformers.modeling_outputs import CausalLMOutput
from torchmetrics import CharErrorRate, WordErrorRate


In [ ]:
class CNNLayerNorm(nn.Module):
    """Layer normalization built for cnns input"""
    def __init__(self, n_feats):
        super(CNNLayerNorm, self).__init__()
        self.layer_norm = nn.LayerNorm(n_feats)

    def forward(self, x):
        x = x.transpose(2, 3).contiguous()  # (batch, channel, time, feature)
        x = self.layer_norm(x)
        return x.transpose(2, 3).contiguous()  # (batch, channel, feature, time)


class ResidualCNN(nn.Module):
    """Residual CNN inspired by the model architecture in DeepSpeech 2."""
    def __init__(self, in_channels, out_channels, kernel, stride, dropout, n_feats):
        super(ResidualCNN, self).__init__()

        self.cnn1 = nn.Conv2d(in_channels, out_channels, kernel, stride, padding=kernel//2)
        self.cnn2 = nn.Conv2d(out_channels, out_channels, kernel, stride, padding=kernel//2)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.layer_norm1 = CNNLayerNorm(n_feats)
        self.layer_norm2 = CNNLayerNorm(n_feats)

    def forward(self, x):
        residual = x  # (batch, channel, feature, time)
        x = self.layer_norm1(x)
        x = nn.functional.gelu(x)
        x = self.dropout1(x)
        x = self.cnn1(x)
        x = self.layer_norm2(x)
        x = nn.functional.gelu(x)
        x = self.dropout2(x)
        x = self.cnn2(x)
        x += residual
        return x


class BidirectionalGRU(nn.Module):
    """Bidirectional GRU with layer normalization and dropout."""
    def __init__(self, rnn_dim, hidden_size, dropout, batch_first):
        super(BidirectionalGRU, self).__init__()

        self.BiGRU = nn.GRU(
            input_size=rnn_dim, hidden_size=hidden_size,
            num_layers=1, batch_first=batch_first, bidirectional=True)
        self.layer_norm = nn.LayerNorm(rnn_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.layer_norm(x)
        x = nn.functional.gelu(x)
        x = self.dropout(x)
        x, _ = self.BiGRU(x)
        x = self.dropout(x)
        return x


In [ ]:
class FeatureExtractor(nn.Module):
    """Extracts features for ASR using CNNs and Residual connections."""

    def __init__(self, n_feats, n_cnn_layers, out_channels=32, dropout=0.1):
        super(FeatureExtractor, self).__init__()
        self.initial_cnn = nn.Conv2d(1, out_channels, kernel_size=3, stride=2, padding=1)

        self.residual_cnn_layers = nn.Sequential(*[
            ResidualCNN(out_channels, out_channels, kernel=3, stride=1, dropout=dropout, n_feats=n_feats)
            for _ in range(n_cnn_layers)
        ])

    def forward(self, x):
        x = self.initial_cnn(x)
        x = self.residual_cnn_layers(x)
        return x


In [ ]:
class SpeechRecognitionModel(nn.Module):
    """Speech Recognition Model combining feature extraction, bidirectional GRUs, and a classifier."""

    def __init__(self, n_feats, n_class, rnn_dim, n_rnn_layers, n_cnn_layers, dropout=0.1):
        super(SpeechRecognitionModel, self).__init__()
        self.feature_extractor = FeatureExtractor(n_feats=n_feats//2, n_cnn_layers=n_cnn_layers, out_channels=32, dropout=dropout)

        self.fully_connected = nn.Linear(n_feats * 16, rnn_dim)  # Adjust the multiplication factor based on the final output size
        self.rnn_layers = nn.Sequential(*[
            BidirectionalGRU(rnn_dim=rnn_dim if i == 0 else rnn_dim * 2, hidden_size=rnn_dim, dropout=dropout, batch_first=True)
            for i in range(n_rnn_layers)
        ])
        self.classifier = nn.Sequential(
            nn.Linear(rnn_dim * 2, rnn_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(rnn_dim, n_class)
        )

    def forward(self, x):
        x = self.feature_extractor(x)
        sizes = x.size()
        x = x.view(sizes[0], sizes[1] * sizes[2], sizes[3])  # Reshape for RNN layers
        x = x.transpose(1, 2)  # (batch, time, feature)
        x = self.fully_connected(x)
        x = self.rnn_layers(x)
        x = self.classifier(x)
        return x


In [ ]:
def wrapped_generate(model_to_wrap: nn.Module, **kwargs) -> torch.Tensor:
    """Generates output sequences using greedy decoding.

    Parameters
    ----------
    model_to_wrap : nn.Module
        The ASR model to be used for generating the output.
    kwargs
        Arguments to be passed to the model for generating output.

    Returns
    -------
    torch.Tensor
        The generated output sequences in the form of torch.Tensor.
    """
    input_tensor = kwargs.get('input_tensor')
    blank_label = kwargs.get('blank_label', 28)  # Default blank label

    model_to_wrap.eval()
    with torch.no_grad():
        logits = model_to_wrap(input_tensor)  # Obtain logits from the model
        logits = logits.transpose(0, 1)  # Transpose to (time, batch, n_class)
        probabilities = torch.softmax(logits, dim=-1)
        best_paths = torch.argmax(probabilities, dim=-1)

    decoded_sequences = []
    max_length = 0
    for path in best_paths:
        sequence = []
        previous = None
        for label in path:
            if label != previous and label != blank_label:
                sequence.append(label.item())
            previous = label
        max_length = max(max_length, len(sequence))
        decoded_sequences.append(sequence)

    # Pad sequences to the same length
    padded_sequences = [seq + [blank_label]*(max_length - len(seq)) for seq in decoded_sequences]

    return torch.tensor(padded_sequences)


In [ ]:
import torch
import pytorch_lightning as pl
from torchmetrics import CharErrorRate, WordErrorRate
from torch.nn import CTCLoss
from dataclasses import dataclass
from typing import List

from pytorch_lightning.utilities.types import STEP_OUTPUT



class AutomaticSpeechRecognitionTask(pl.LightningModule):
    def __init__(self, model: nn.Module, vocab: Vocabulary, learning_rate: float = 0.001):
        super(AutomaticSpeechRecognitionTask, self).__init__()
        self.model = model
        self.learning_rate = learning_rate
        self.vocab = vocab

        self.cer = CharErrorRate()
        self.wer = WordErrorRate()
        self.criterion = CTCLoss(blank=self.vocab.pad_id())

    def training_step(self, batch: ASRBatch, batch_idx: int) -> STEP_OUTPUT:
        spectrograms = batch.spectrograms
        labels = batch.labels
        input_lengths = batch.attention_mask.sum(dim=-1)
        label_lengths = batch.label_mask.sum(dim=-1)

        spectrograms, labels = spectrograms.to(self.device), labels.to(self.device)

        logits = self.model(spectrograms).transpose(0, 1)  # (time, batch, n_class)
        loss = self.criterion(logits, labels, input_lengths, label_lengths)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch: ASRBatch, batch_idx: int):
        spectrograms = batch.spectrograms
        labels = batch.labels
        input_lengths = batch.attention_mask.sum(dim=-1)
        label_lengths = batch.label_mask.sum(dim=-1)

        spectrograms, labels = spectrograms.to(self.device), labels.to(self.device)

        logits = self.model(spectrograms).transpose(0, 1)  # (time, batch, n_class)
        loss = self.criterion(logits, labels, input_lengths, label_lengths)
        self.log('val_loss', loss, on_epoch=True, prog_bar=True, logger=True)

        # Decode each sequence in the batch using wrapped_generate
        decoded_sequences = [wrapped_generate(model_to_wrap=self.model, input_tensor=spec.unsqueeze(0), blank_label=self.vocab.pad_id())[0] for spec in spectrograms]

        # Convert decoded sequences to strings
        decoded_strs = [''.join([self.vocab.idx2token(idx) for idx in sequence]) for sequence in decoded_sequences]

        # Convert label sequences to strings using idx2token
        label_strs = [''.join([self.vocab.idx2token(idx) for idx in label_seq.tolist()]) for label_seq in labels]

        # Calculate and log CER and WER
        curr_cer = self.cer(decoded_strs, label_strs)
        curr_wer = self.wer(decoded_strs, label_strs)
        self.log('val_cer', curr_cer, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_wer', curr_wer, on_epoch=True, prog_bar=True, logger=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)


In [ ]:
import pytorch_lightning as pl
import logging
from pytorch_lightning.profiler import SimpleProfiler, AdvancedProfiler

logging.getLogger("pytorch_lightning").setLevel(logging.INFO)
profiler = AdvancedProfiler()

# Configure data module
asr_datamodule = SpeechRecognitionDataModule(
    datasets={
        'train': train_dataset,
        'val': dev_dataset
    },
    vocab=vocab,
    batch_size=4  # Assuming the batch size used in training
)

asr_model = SpeechRecognitionModel(
    n_feats= 64,  # Assuming 128-dimensional MelSpectrogram features
    n_class=len(vocab),  # Number of classes based on the vocabulary size
    rnn_dim=512,  # Assuming the RNN dimension used in training
    n_rnn_layers=5,  # Assuming the number of RNN layers used in training
    n_cnn_layers=3,  # Assuming the number of CNN layers used in training
    dropout=0.1  # Dropout rate
)

# Check if the code is running in autograder mode or not
if is_autograder:
    # Assumes a checkpoint URL is provided for autograder mode
    CHECKPOINT_TO_DOWNLOAD = ...
    from onedrivedownloader import download as onedrive_download
    onedrive_download(CHECKPOINT_TO_DOWNLOAD, filename='checkpoint.ckpt', unzip=False)
    asr_pl_module = AutomaticSpeechRecognitionTask.load_from_checkpoint(
        checkpoint_path='checkpoint.ckpt',
        vocab=vocab,
        model=asr_model
    )
else:
    # Initialize the ASR task for training
    asr_pl_module = AutomaticSpeechRecognitionTask(
        vocab=vocab,
        model=asr_model,
        learning_rate=5e-4  # Assuming the learning rate used in training
    )

    # Configure the trainer
    asr_trainer = pl.Trainer(
        accelerator=accelerator,
        min_epochs=10,  # Minimum number of epochs
        max_epochs=20,  # Maximum number of epochs
        profiler=profiler,
        log_every_n_steps=1,
        callbacks=[pl.callbacks.EarlyStopping(monitor='val_cer', mode='min')]
    )

    # Start training
    asr_trainer.fit(model=asr_pl_module, datamodule=asr_datamodule)
    print(profiler.summary())



INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type                   | Params
-----------------------------------------------------
0 | model     | SpeechRecognitionModel | 23.2 M
1 | cer       | CharErrorRate          | 0     
2 | wer       | WordErrorRate          | 0     
3 | criterion | CTCLoss                | 0     
-----------------------------------------------------
23.2 M    Trainable params
0         Non-trainable params
23.2 M    Total params
92.729    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/data.py:98: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 4. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the 

Training: 0it [00:00, ?it/s]

The code below (the next 4 classes, Feautre Extractor, Speech recognition, wrapped generate and automatic speech recognition task) have all been modified. The other thing thats been modified is the data initialization in the training part.

In [ ]:
import torch
import torch.nn as nn
from typing import Optional
from transformers.modeling_outputs import CausalLMOutput

class FeatureExtractor(nn.Module):
    def __init__(self, n_feats, n_features, output_size=64):
        super(FeatureExtractor, self).__init__()
        # Convolutional layer
        self.conv = nn.Conv2d(1, n_features, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu = nn.ReLU()

        # Calculate the size after the convolution
        conv_height = n_feats // 2  # This is an assumption; adjust based on your actual input size

        # Linear layer to get the output to the desired size
        self.linear = nn.Linear(n_features * conv_height, output_size)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.conv(x)
        x = self.relu(x)

        # Flatten and pass through the linear layer
        x = x.permute(0, 2, 3, 1)  # Rearrange to (batch_size, height, width, channels)
        x = x.view(x.size(0), -1)  # Flatten height and channels
        x = self.linear(x)
        return x

class SpeechRecognitionModel(nn.Module):
    def __init__(self, vocab, n_feats, n_features, rnn_dim, n_rnn_layers, dropout):
        super(SpeechRecognitionModel, self).__init__()
        self.vocab = vocab
        self.feature_extractor = FeatureExtractor(n_feats, n_features, output_size=n_feats)

        # Bidirectional GRU
        self.rnn = nn.GRU(input_size=n_feats, hidden_size=rnn_dim, num_layers=n_rnn_layers, dropout=dropout, batch_first=True, bidirectional=True)

        # Linear layer to map RNN outputs to token predictions
        self.lm_head = nn.Linear(rnn_dim * 2, len(vocab))  # *2 for bidirectional

    def forward(self, input_values, attention_mask=None, labels=None, label_mask=None):
        # Process input through feature extractor and RNN
        x = self.feature_extractor(input_values)
        x, _ = self.rnn(x)
        logits = self.lm_head(x)

        # Compute CTC loss if labels are provided
        loss = None
        if labels is not None and label_mask is not None:
            log_probs = nn.functional.log_softmax(logits, dim=-1).transpose(0, 1)
            input_lengths = torch.full((x.size(0),), x.size(1), dtype=torch.long)
            target_lengths = label_mask.sum(-1)

            flattened_targets = labels.masked_select(label_mask)
            loss = nn.functional.ctc_loss(
                log_probs=log_probs,
                targets=flattened_targets,
                input_lengths=input_lengths,
                target_lengths=target_lengths,
                blank=self.vocab.pad_id(),
                zero_infinity=True  # To handle potential infinities in the loss
            )

        return CausalLMOutput(loss=loss, logits=logits)


In [ ]:
def wrapped_generate(model_to_wrap: nn.Module, input_values: torch.Tensor, vocab: Vocabulary) -> List[str]:
    """
    Uses the model to generate predictions and decodes them into strings using
    the provided Vocabulary's decoding function.

    Parameters
    ----------
    model_to_wrap : nn.Module
        The ASR model to generate predictions.
    input_values : torch.Tensor
        The input tensor containing spectrogram features.
    vocab : Vocabulary
        The Vocabulary object for decoding.

    Returns
    -------
    List[str]
        The list of decoded strings.
    """
    model_to_wrap.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        # Forward pass through the model
        output = model_to_wrap(input_values=input_values)
        logits = output.logits

        # Greedy decoding: choose the most probable token at each time step
        predicted_ids = torch.argmax(logits, dim=-1)

        # Use the Vocabulary's decoding function
        decoded_outputs = decode_as_str(vocab, predicted_ids)

    return decoded_outputs

# Example usage:
# decoded_strings = wrapped_generate(asr_model, input_spectrogram_tensor, vocab)


In [ ]:
from pytorch_lightning import LightningModule
from torchmetrics import CharErrorRate, WordErrorRate

class AutomaticSpeechRecognitionTask(LightningModule):
    def __init__(self, model: nn.Module, vocab: Vocabulary, learning_rate: float = 0.001):
        super(AutomaticSpeechRecognitionTask, self).__init__()
        self.model = model
        self.learning_rate = learning_rate
        self.vocab = vocab

        # Metrics
        self.cer = CharErrorRate()
        self.wer = WordErrorRate()

    def forward(self,
                input_values: torch.Tensor,
                attention_mask: Optional[torch.Tensor] = None,
                labels: Optional[torch.Tensor] = None,
                label_mask: Optional[torch.Tensor] = None):
        return self.model(input_values, attention_mask, labels, label_mask)

    def training_step(self, batch: ASRBatch, batch_idx: int):
        # Access data from batch using attributes
        input_values = batch.spectrograms
        labels = batch.labels
        attention_mask = batch.attention_mask
        label_mask = batch.label_mask
        # label_strs = batch.label_strs  # Not needed for training_step

        # Forward pass
        outputs = self(input_values, attention_mask, labels, label_mask)
        loss = outputs.loss

        # Logging
        batch_size = input_values.size(0)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True, batch_size=batch_size)
        return loss

    def validation_step(self, batch: ASRBatch, batch_idx: int):
        input_values = batch.spectrograms
        labels = batch.labels
        attention_mask = batch.attention_mask
        label_mask = batch.label_mask
        label_strs = batch.label_strs

        outputs = self(input_values, attention_mask, labels, label_mask)
        logits = outputs.logits

        # Greedy decoding and metrics calculation
        predicted_ids = torch.argmax(logits, dim=-1)
        decoded_outputs = decode_as_str(self.vocab, predicted_ids)
        curr_cer = self.cer(decoded_outputs, label_strs)
        curr_wer = self.wer(decoded_outputs, label_strs)

        # Logging
        batch_size = input_values.size(0)
        self.log('val_cer', curr_cer, on_epoch=True, prog_bar=True, batch_size=batch_size)
        self.log('val_wer', curr_wer, on_epoch=True, prog_bar=True, batch_size=batch_size)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer


In [ ]:
# Configure data module
asr_datamodule = SpeechRecognitionDataModule(
    datasets={
        'train': train_dataset,  # Your training dataset
        'val': dev_dataset       # Your validation dataset
    },
    vocab=vocab,
    batch_size=64  # Batch size, adjust as needed
)

# Parameters for the model
n_feats = 64  # Number of Mel-spectrogram features
n_features = 64  # Adjusted to match the output of the FeatureExtractor
rnn_dim = 512  # Hidden size of RNN layers
n_rnn_layers = 3  # Number of RNN layers
dropout = 0.1  # Dropout rate

# Instantiate the ASR model
asr_model = SpeechRecognitionModel(
    vocab=vocab,
    n_feats=n_feats,
    n_features=n_features,
    rnn_dim=rnn_dim,
    n_rnn_layers=n_rnn_layers,
    dropout=dropout
)

# Create the ASR task and train the model
if is_autograder:
    # Download and load the checkpoint for the autograder
    CHECKPOINT_TO_DOWNLOAD = ...  # Replace with your checkpoint URL
    onedrive_download(CHECKPOINT_TO_DOWNLOAD, filename='checkpoint.ckpt', unzip=False)
    asr_pl_module = AutomaticSpeechRecognitionTask.load_from_checkpoint(
        checkpoint_path='checkpoint.ckpt',
        vocab=vocab,
        model=asr_model
    )
else:
    # Create a new task for training in student mode
    asr_pl_module = AutomaticSpeechRecognitionTask(
        model=asr_model,
        vocab=vocab,
        learning_rate=5e-4
    )

    # Configure the PyTorch Lightning Trainer
    asr_trainer = pl.Trainer(
        accelerator=accelerator,
        min_epochs=10,
        max_epochs=20,
        callbacks=[pl.callbacks.EarlyStopping(monitor='val_cer', mode='min')]
    )

    # Train the model
    asr_trainer.fit(asr_pl_module, asr_datamodule)


Another old imp

In [ ]:
import torch
import torch.nn as nn
from transformers.modeling_outputs import CausalLMOutput

class FeatureExtractor(nn.Module):
    def __init__(self, n_mels: int, n_features: int):
        super(FeatureExtractor, self).__init__()
        self.conv = nn.Conv2d(1, n_features, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.bn = nn.BatchNorm2d(n_features)
        self.relu = nn.ReLU()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x.transpose(1, -1)  # (batch_size, time, mapped_feature)

class SpeechRecognitionModel(nn.Module):
    def __init__(self, vocab: Vocabulary, n_feats: int, rnn_dim: int, n_rnn_layers: int):
        super(SpeechRecognitionModel, self).__init__()
        self.vocab = vocab
        self.feature_extractor = FeatureExtractor(n_mels=n_feats, n_features=n_feats)

        # Store rnn_dim and n_rnn_layers as instance attributes
        self.rnn_dim = rnn_dim
        self.n_rnn_layers = n_rnn_layers

        # Placeholder for the LSTM input size and the LSTM layer
        self.rnn = nn.LSTM(input_size=2048, hidden_size=rnn_dim, num_layers=n_rnn_layers, batch_first=True)
        self.fc = nn.Linear(rnn_dim, len(vocab))

    def forward(self, input_values, attention_mask=None, labels=None, label_mask=None):

        input_values = input_values.cuda()
        attention_mask = attention_mask.cuda() if attention_mask is not None else None
        labels = labels.cuda() if labels is not None else None
        label_mask = label_mask.cuda() if label_mask is not None else None
        features = self.feature_extractor(input_values)
        batch_size, time, channels, feat_dim = features.size()



        features = features.reshape(batch_size, time, channels * feat_dim)
        rnn_out, _ = self.rnn(features)
        logits = self.fc(rnn_out)

        loss = None
        if labels is not None:
            log_probs = nn.functional.log_softmax(logits, dim=-1).transpose(0, 1)
            input_lengths = torch.full(size=(log_probs.size(1),), fill_value=log_probs.size(0), dtype=torch.long)
            target_lengths = label_mask.sum(-1)
            flattened_targets = labels.masked_select(label_mask)
            loss = nn.functional.ctc_loss(log_probs, flattened_targets, input_lengths, target_lengths, blank=self.vocab.pad_id(), zero_infinity=True)

        return CausalLMOutput(loss=loss, logits=logits)

# Assuming Vocabulary and other necessary classes are defined elsewhere


In [ ]:
def wrapped_generate(model_to_wrap: nn.Module, input_values: torch.Tensor, vocab: Vocabulary) -> List[str]:
    model_to_wrap.eval()
    with torch.no_grad():
        logits = model_to_wrap(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    decoded_outputs = [decode_as_str(vocab, ids) for ids in predicted_ids]
    return decoded_outputs


In [ ]:
from torchmetrics import CharErrorRate, WordErrorRate

class AutomaticSpeechRecognitionTask(pl.LightningModule):
    def __init__(self, model: nn.Module, vocab: Vocabulary, learning_rate: float = 0.001):
        super(AutomaticSpeechRecognitionTask, self).__init__()
        self.model = model
        self.learning_rate = learning_rate
        self.vocab = vocab

        self.cer = CharErrorRate()
        self.wer = WordErrorRate()

    def training_step(self, batch: ASRBatch):
      print(f"Model device: {next(self.model.parameters()).device}")
      print(f"Input device: {batch.spectrograms.device}")

      batch.spectrograms = batch.spectrograms.cuda()
      batch.attention_mask = batch.attention_mask.cuda()
      batch.labels = batch.labels.cuda()
      batch.label_mask = batch.label_mask.cuda()
      outputs = self.model(
          input_values=batch.spectrograms,
          attention_mask=batch.attention_mask,
          labels=batch.labels,
          label_mask=batch.label_mask
      )
      self.log('train_loss', outputs.loss)
      return outputs.loss

    def validation_step(self, batch: ASRBatch, batch_idx: int):
        print(f"Model device: {next(self.model.parameters()).device}")
        print(f"Input device: {batch.spectrograms.device}")
        batch.spectrograms = batch.spectrograms.cuda()
        batch.attention_mask = batch.attention_mask.cuda()
        outputs = self.model(
            input_values=batch.spectrograms,
            attention_mask=batch.attention_mask,
        )
        logits = outputs.logits
        predicted_ids = torch.argmax(logits, dim=-1)
        decoded_outputs = [decode_as_str(self.vocab, ids) for ids in predicted_ids]
        curr_cer = self.cer(decoded_outputs, batch.label_strs)
        curr_wer = self.wer(decoded_outputs, batch.label_strs)
        self.log('val_cer', curr_cer, on_step=False, on_epoch=True)
        self.log('val_wer', curr_wer, on_step=False, on_epoch=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer


In [ ]:
asr_datamodule = SpeechRecognitionDataModule(
    datasets={
        'train': train_dataset,
        'val': dev_dataset
    },
    vocab=vocab,
    batch_size=64
)

asr_model = SpeechRecognitionModel(
    vocab=vocab,
    n_feats=64,  # Number of Mel-frequency bands
    rnn_dim=368,  # Dimensionality of the RNN
    n_rnn_layers=3  # Number of RNN layers
)

# Check if running in autograder mode
if is_autograder:
    # Download the checkpoint and load it
    # Code for downloading and loading the checkpoint
    pass
else:
    # If not in autograder mode, train the model
    asr_pl_module = AutomaticSpeechRecognitionTask(
        model=asr_model,
        vocab=vocab,
        learning_rate=5e-4
    )
    asr_trainer = pl.Trainer(
        accelerator=accelerator,
        min_epochs=10,
        max_epochs=20,
        callbacks=[pl.callbacks.EarlyStopping(monitor='val_cer', mode='min')]
    )
    asr_trainer.fit(model=asr_pl_module, datamodule=asr_datamodule)


In [ ]:
!pip install torchmetrics==0.11.4


In [ ]:
import torchmetrics

This is the old imp

In [ ]:
class FeatureExtractor(nn.Module):
    """Extracts features for ASR.

    If needed, you have to use your auxiliary modules to extract features.
    Please modify the `__init__` to make it compatible with your model designs.
    """

    def __init__(
            self,
            numFeatures
    ):
        super(FeatureExtractor, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv2 = nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv3 = nn.Conv2d(64, numFeatures, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.relu = nn.ReLU()

        # Your code here
        ...

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """This forward method processes the input spectrogram features and returns
        a tensor that represents transformed features.

        Please pay attention to the output tensor shape, you have to use `transpose`
        to make sure dimensions aligned.

        Parameters
        ----------
        x : torch.Tensor
            The input spectrograms, in the shape of (batch_size, channel, feature, time)

        Returns
        -------
        torch.Tensor
            (batch_size, time, mapped_feature)

        """
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))

        x = x.transpose(1, 2).transpose(2, 3).squeeze(1)
        print(x.shape)
        return x

        ...


class SpeechRecgonitionModel(nn.Module):
    def __init__(self, vocab, numFeatures, dimRnn, noLayers, dropout):
        super(SpeechRecgonitionModel, self).__init__()
        self.vocab = vocab
        self.featureExtractor = FeatureExtractor(numFeatures)
        self.rnn = nn.LSTM(input_size=numFeatures, hidden_size=dimRnn, num_layers=noLayers, dropout=dropout, batch_first=True)
        self.fullConn = nn.Linear(dimRnn, len(vocab))

    def forward(self, input_values, attention_mask=None, labels=None, label_mask=None):
        features = self.featureExtractor(input_values)
        print("Feature Shape:", features.shape)

        # Add this line to check the shape before the LSTM
        rnn_input = features.unsqueeze(1)
        print("RNN Input Shape:", rnn_input.shape)

        loss = None
        if labels is not None and label_mask is not None:
            target_lengths = label_mask.sum(-1)
            flattened_targets = labels.masked_select(label_mask)
            log_probs = nn.functional.log_softmax(logits, dim=-1).transpose(0, 1)

            # Please make sure this aligns with the number of conv kernel
            if attention_mask is not None:
                input_lengths = attention_mask.sum(-1)
            else:
                input_lengths = torch.full((logits.size(0),), logits.size(1), dtype=torch.long, device=logits.device)

            loss = nn.functional.ctc_loss(
                log_probs=log_probs,
                targets=flattened_targets,
                input_lengths=input_lengths,
                target_lengths=target_lengths,
                blank=self.vocab.pad_id()
            )

        return CausalLMOutput(loss=loss, logits=logits)

        # Please make sure that you are returning a `CausalLMOutput`.
        # Your code here
        ...

In [ ]:
def GreedyDecoder(output, labels, labelLengths, blankLabel, textTransform, collapseRepeated=True):
    argMaxes = torch.argmax(output, dim=2)
    decodes = []
    targets = []
    for i, args in enumerate(argMaxes):
        decode = []
        targets.append(textTransform.int_to_text(labels[i][:labelLengths[i]].tolist()))
        for j, index in enumerate(args):
            if index != blankLabel:
                if collapseRepeated and j != 0 and index == args[j - 1]:
                    continue
                decode.append(index.item())
        decodes.append(textTransform.int_to_text(decode))
    return decodes, targets

In [ ]:
def wrapped_generate(model_to_wrap: nn.Module, **kwargs) -> torch.Tensor:
    """Wraps the generate method. This function is what will be actually
    called by the evaluation routine for the leaderboard.

    In this function, you can wrap your `generate()` method to allow
    different generation configurations to be used at the test time.

    Parameters
    ----------
    model_to_wrap : nn.Module
        Your ASR model.
    kwargs
        Argument dict that passes all arguments to the generate function.

    Returns
    -------
        Generated phoneme sequences in the form of torch.Tensor.
        In the shape of (batch_size, time).
    """
    # Please implement your decoding strategy here.
    # You have to pass inputs to the model and run the model once (in no_grad mode)
    # and decode with your decoding strategy to get the tensor of token ids
    # The simplest one is the greedy decoding - recall that we did it in hw2
    # It is also the one used in the `validation_step`.
    # You might see multiple repeated characters. In this case, you could also implement
    # some heuristics in this function to do post-processing to remove
    # repeated nonsensical characters.
    model_to_wrap.eval()

    with torch.no_grad():
        inputValues = kwargs.get('input_values')
        labels = kwargs.get('labels')
        labelLengths = kwargs.get('label_lengths')
        textTransform = kwargs.get('text_transform')
        blankLabel = kwargs.get('blank_label', 28)
        if inputValues is None:
            raise ValueError("Input values are required for generation")

        outputs = model_to_wrap(inputValues)

        # Assuming the model outputs logits directly
        logits = outputs.logits if hasattr(outputs, 'logits') else outputs

        # Apply Greedy Decoding
        decodes, _ = GreedyDecoder(
            logits,
            labels,
            labelLengths,
            blankLabel,
            textTransform,
            collapse_repeated=True
        )

        # Convert decoded sequences to tensor
        decodedSeq = [torch.tensor(textTransform.text_to_int(text)) for text in decodes]
        decodedSeqPad = torch.nn.utils.rnn.pad_sequence(decodedSeq, batch_first=True)

    return decodedSeqPad
    ...

In [ ]:
from torchmetrics import CharErrorRate, WordErrorRate
from pytorch_lightning.utilities.types import STEP_OUTPUT


class AutomaticSpeechRecognitionTask(pl.LightningModule):
    def __init__(self,
                 model: nn.Module,
                 vocab: Vocabulary,
                 learning_rate: float = 0.001):
        super(AutomaticSpeechRecognitionTask, self).__init__()
        self.model = model
        self.learning_rate = learning_rate
        self.vocab = vocab

        self.cer = CharErrorRate()
        self.wer = WordErrorRate()

    def training_step(self, batch: ASRBatch) -> STEP_OUTPUT:
        """Defines the training step.

        Parameters
        ----------
        batch : ASRBatch
            The batched training instances.

        Returns
        -------
        loss : torch.Tensor
            The loss computed from the ASR model.
        """
        # Please make necessary modifications to accommodate your design
        outputs = self.model(
            input_values=batch.spectrograms,
            attention_mask=batch.attention_mask,
            labels=batch.labels,
            label_mask=batch.label_mask
        )
        return outputs.loss

    def validation_step(self, batch: ASRBatch, batch_idx: int):
        """Defines the validation step - for this module, we have the same
        training and validation behaviors. Usually, we would compute a metric that is
        used to select the best performing model checkpoint.

        Parameters
        ----------
        batch : ASRBatch
            The batched training instances.
        batch_idx: int
            The index of the batch.
        """
        # You are free to modify this function to ensure they are being called correctly.
        outputs = self.model(
            input_values=batch.spectrograms,
            attention_mask=batch.attention_mask,
        )
        logits = outputs.logits
        # take argmax and decode - Using greedy decoding
        predicted_ids = torch.argmax(logits, dim=-1)
        decoded_outputs = decode_as_str(self.vocab, predicted_ids)
        curr_cer = self.cer(decoded_outputs, batch.label_strs)
        curr_wer = self.wer(decoded_outputs, batch.label_strs)
        self.log('val_cer', self.cer)
        self.log('val_wer', self.wer)

    def configure_optimizers(self):
        """Configures optimizers for the training."""
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

In [ ]:
# Configure data module
asr_datamodule = SpeechRecognitionDataModule(
    datasets={
        'train': train_dataset,
        'val': dev_dataset
    },
    vocab=vocab,
    batch_size=64
)
# Please modify the model and trainer configurations to accommodate your design
asr_model = SpeechRecgonitionModel(
    vocab=vocab,
    # The following args might not align with your model!
    numFeatures=128,
    dimRnn=512,
    noLayers=3,
    dropout=0.1
)
if is_autograder:  # You have to make sure that you checkpoint can be correctly loaded by the autograder
    # Please upload your checkpoint to your OneDrive
    # and put the share link below `CHECKPOINT_TO_DOWNLOAD`
    CHECKPOINT_TO_DOWNLOAD = ...
    # Downloads the checkpoint
    from onedrivedownloader import download as onedrive_download
    onedrive_download(CHECKPOINT_TO_DOWNLOAD, filename='checkpoint.ckpt', unzip=False)
    asr_pl_module = AutomaticSpeechRecognitionTask.load_from_checkpoint(
        checkpoint_path='checkpoint.ckpt',
        vocab=vocab,
        model=asr_model
    )
else:
    # In the student mode, a new model would be trained
    # You are allowed to change training hyperparameters
    # But you are not allowed to create a new task
    asr_pl_module = AutomaticSpeechRecognitionTask(
        vocab=vocab,
        model=asr_model,
        learning_rate=5e-4
    )
    asr_trainer = pl.Trainer(
        accelerator=accelerator,
        min_epochs=10,
        max_epochs=20,
        callbacks=[pl.callbacks.EarlyStopping(monitor='val_cer', mode='min')]
    )
    asr_trainer.fit(model=asr_pl_module, datamodule=asr_datamodule)

In [ ]:
torch.cuda.empty_cache()


In [ ]:
asr_trainer.logged_metrics

In [ ]:
if not is_autograder:
    # You can run this cell to test whether you have get your model trained
    # You are expecting to see a phoneme sequence
    train_dataloader = asr_datamodule.train_dataloader()
    test_example = None
    for i, bc in enumerate(train_dataloader):
        if i > 0:
            break
        test_example = bc

    print(decode_as_str(
        asr_pl_module.model.vocab,
        wrapped_generate(
            model_to_wrap=asr_pl_module.model,
            input_values=test_example.spectrograms.to(asr_pl_module.device),
            attention_mask=test_example.attention_mask.to(asr_pl_module.device)
        )
    ))